In [1]:
# Importing the pandas library to work with DataFrames
import pandas as pd
import numpy as np
import pickle
from sklearn.impute import SimpleImputer




In [2]:
!pip install langchain
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pymongo

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')
db = client['diet_recommendations']

users_collection = db['users']
recommendations_collection = db['recommendations']


In [4]:
!pip install openai

In [5]:
!pip install faiss-cpu


In [6]:
!pip install -U langchain-community

In [7]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import os
import openai
import mysql.connector
import datetime
from pymongo import MongoClient
import uuid
from datetime import datetime, timedelta

In [1]:
# Set OpenAI API key
openai.api_key = 'Your openai key'
os.environ["OPENAI_API_KEY"] = openai.api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")



NameError: name 'openai' is not defined

In [9]:
# Load data
txt_file_path = "C:\\Users\\user\\Downloads\\Kenya Recipe Book 2018.txt"
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()

# Summarize data to avoid exceeding token limit
def summarize_data(data, max_length=900000):
    summarized_data = ""
    current_length = 0
    for doc in data:
        if current_length + len(doc.page_content) > max_length:
            break
        summarized_data += doc.page_content + "\n"
        current_length += len(doc.page_content)
    return summarized_data

# Create embeddings and vectorstore
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

# Initialize chat model and memory
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

c:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
# Create conversational retrieval chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)



In [11]:
# Function to calculate BMI
def calculate_bmi(weight, height):
    height_in_meters = height / 100  # Convert height to meters
    bmi = weight / (height_in_meters ** 2)
    return bmi

# Function to get completion from OpenAI
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [12]:
# Add user function
def add_user(age, height, weight, gender, goal, duration, health_issues):
    user_id = str(uuid.uuid4())
    user_data = {
        'user_id': user_id,
        'age': age,
        'height': height,
        'weight': weight,
        'gender': gender,
        'goal': goal,
        'duration': duration,
        'health_issues': health_issues
    }
    users_collection.insert_one(user_data)
    return user_id

# Get past meals function
def get_past_meals(user_id):
    thirty_days_ago = datetime.now() - timedelta(days=30)
    past_meals = recommendations_collection.find({
        'user_id': user_id,
        'date': {'$gte': thirty_days_ago}
    }, {'meal': 1, '_id': 0})
    return [meal['meal'] for meal in past_meals]

In [13]:
from datetime import datetime

# Function for daily recommendation
def daily_recommendation(user_id, data_text):
    today = datetime.now().date()
    today_str = today.strftime('%Y-%m-%d')

    existing_meal = recommendations_collection.find_one({
        'user_id': user_id,
        'date': today_str
    }, {'meal': 1, '_id': 0})

    if existing_meal:
        meal = existing_meal['meal']
    else:
        meal = generate_personalized_diet(user_id, today_str, data_text)
    
    return meal



In [14]:
def calculate_kilojoules_intake(age, height, weight, gender, goal, activity_level='sedentary'):
    # Calculate BMR
    if gender == 'male':
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    elif gender == 'female':
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    else:
        raise ValueError("Gender must be 'male' or 'female'")

    # Adjust BMR for activity level
    activity_multipliers = {
        'sedentary': 1.2,
        'lightly_active': 1.375,
        'moderately_active': 1.55,
        'very_active': 1.725,
        'super_active': 1.9
    }

    if activity_level not in activity_multipliers:
        raise ValueError("Invalid activity level. Choose from: sedentary, lightly_active, moderately_active, very_active, super_active")

    total_kilojoules = bmr * activity_multipliers[activity_level]

    
    if goal > weight:  # Weight gain goal
        total_kilojoules += 500  # Add extra kilojoules for weight gain
    elif goal < weight:  # Weight loss goal
        total_kilojoules -= 500  # Subtract kilojoules for weight loss

    return total_kilojoules


In [15]:
def generate_personalized_diet(user_id, date, data_text):
    user_data = users_collection.find_one({'user_id': user_id})

    if not user_data:
        raise ValueError(f"User with ID {user_id} does not exist.")
    
    age, height, weight, gender, goal, health_issues = user_data['age'], user_data['height'], user_data['weight'], user_data['gender'], user_data['goal'], user_data['health_issues']
    bmi = calculate_bmi(weight, height)
    
    # Calculate daily kilojoules intake
    kilojoules_needed = calculate_kilojoules_intake(age, height, weight, gender, goal)

    # Get past meals
    past_meals = get_past_meals(user_id)
    
    prompt =f"""
     User Profile:
     - Age: {age}
     - Height: {height} cm
     - Weight: {weight} kg
     - Gender: {gender}
     - Weight Goal: {goal} kg
     - Health Issues: {health_issues}
     - BMI: {bmi}
     - Daily Kilojoules Intake: {kilojoules_needed} kJ

     Recent Meal History:
    {', '.join(past_meals)}

     Dietary Requirements:
     1. Use only recipes from the Kenya Recipe Book 2018
     2. Align with weight {goal} objective
     3. Avoid allergens and foods related to {health_issues}
     4. Match daily kilojoule intake to {kilojoules_needed} kJ
     5. Ensure variety and avoid repeating meals from the past month

Meal Plan Guidelines:
1. Create a detailed 1-day schedule including:
   - Breakfast
   - Lunch
   - Dinner
   - Snacks
2. Provide portion sizes and timings for each meal
3. Compare nutritional data per 100g of recipe
4. Ensure no consecutive meal repetitions

Formatting:
[Meal Type]: [Meal Name] - [Portion Size] at [Time]

Additional Notes:
- Do not mention the Kenya Recipe Book 2018 explicitly
- Prioritize variety in meal selections

Data Summary:
{data_text[:80000]}

Please generate a 1-day meal plan based on the above information and guidelines.
"""
    
    response = get_completion(prompt)
    
    recommendations_collection.insert_one({
        'user_id': user_id,
        'date': date,
        'meal': response
    })
    
    return response



def calculate_bmi(weight, height):
    return weight / (height / 100) ** 2



In [16]:
# Summarize and prepare the data text
data_text = summarize_data(data)

In [17]:
# Save the model data to a pickle file
model_data = {
    'data_text': data_text,
    # Exclude functions from pickle file
}

with open('updated_diet_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

In [18]:
# Example usage

# Add a new user 
new_user_id = add_user(25, 175, 70, 'male', 50, 12, 'does not eat oatmeal')

# Verify that the user was added
user_data = users_collection.find_one({'user_id': new_user_id})
if not user_data:
    print(f"User with ID {new_user_id} does not exist after insertion.")
else:
    print(f"User with ID {new_user_id} successfully added.")

# Summarize and prepare the data text
data_text = summarize_data(data)

# Get the daily meal plan for the new user
try:
    daily_meal_plan = daily_recommendation(new_user_id, data_text)
    print(daily_meal_plan)
except ValueError as e:
    print(e)


User with ID 18579a59-fce2-4615-b302-0b63a94cf76e successfully added.
Breakfast: Tosti Mayai (Egg Toast) - 2 slices
Time: 7:00 AM

Lunch: Vegetable Samosa - 2 pieces
Time: 12:30 PM

Dinner: Mkate Kuta - 2 pieces
Time: 6:30 PM

Snack: Vimumunya vya Chumvi (Pumpkin & Coconut Milk) - 1 cup
Time: 3:00 PM

Portion sizes and timings have been provided for each meal to align with the user's weight goal and daily kilojoule intake. The meal plan ensures variety and avoids repeating meals from the past month.


In [19]:
# Example usage

# Add a new user 
new_user_id2 = add_user(25, 175, 70, 'male', 80, 12, 'lactose_intolerance')

# Summarize and prepare the data text
data_text = summarize_data(data)

# Get the daily meal plan for the new user
try:
    daily_meal_plan = daily_recommendation(new_user_id2, data_text)
    print(daily_meal_plan)
except ValueError as e:
    print(e)

Breakfast: Oatmeal
- Portion Size: 1 cup
- Time: Morning

Lunch: Vegetable Samosa
- Portion Size: 2 samosas
- Time: Midday

Dinner: Meat Samosa
- Portion Size: 2 samosas
- Time: Evening

Snack: Drop Scones
- Portion Size: 2 scones
- Time: Afternoon

Nutritional Data Comparison per 100g of Recipe:
- Oatmeal: Energy 351 kJ/ 83 kcal, Fat 1.7g, Carbohydrates 12.9 g, Protein 2.4 g, Fibre 3.5 g
- Vegetable Samosa: Energy 1,361 kJ/ 325 kcal, Fat 13.1 g, Carbohydrates 40.2 g, Protein 8.5 g, Fibre 6.2 g
- Meat Samosa: Energy 1,854 kJ/ 443 kcal, Fat 22.2 g, Carbohydrates 40.5 g, Protein 18.8 g, Fibre 3.1 g
- Drop Scones: Energy 1,869 kJ/ 449 kcal, Fat 29.8 g, Carbohydrates 37.4 g, Protein 7.1 g, Fibre 1.4 g

Additional Notes:
- Ensure to follow the portion sizes and timings for each meal
- Adjust the meal plan according to personal preferences and dietary needs
- Stay hydrated throughout the day by drinking water or herbal tea.


In [20]:
# Example usage

# Add a new user 
new_user_id3 = add_user(22, 150, 150, 'female', 80, 30, 'peanuts allergy')

# Summarize and prepare the data text
data_text = summarize_data(data)

# Get the daily meal plan for the new user
try:
    daily_meal_plan = daily_recommendation(new_user_id3, data_text)
    print(daily_meal_plan)
except ValueError as e:
    print(e)

[Breakfast]: Oatmeal - 1 cup at 8:00 AM
[Lunch]: Vegetable Samosa - 2 pieces at 12:00 PM
[Dinner]: Enriched Mandazi - 1 piece at 6:00 PM
[Snack]: Tosti Mayai - 1 slice at 3:00 PM

Note: Ensure to follow the recipe guidelines and adjust portion sizes to meet the daily kilojoule intake of 2144.8995999999993 kJ. Remember to avoid allergens and peanuts due to the user's allergy.
